In [ ]:
!pip install selenium

In [ ]:
!apt-get update

In [ ]:
!apt-get install -y chromium-browser

In [ ]:
!apt install chromium-chromedriver

In [ ]:
!ls /usr/lib/chromium-browser

In [ ]:
import requests
import time
import csv
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

In [ ]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

In [ ]:
def pagesource(url, driver):
    driver = driver
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return soup

In [ ]:
driver = driversetup()
url = 'https://www.bolsasymercados.es/bme-exchange/es/Indices/Resumen'
soup= pagesource (url, driver)
#print(soup.prettify())

In [ ]:
driver = driversetup()
url = 'https://www.bolsasymercados.es/bme-exchange/es/Indices/Resumen'
driver.get(url)
time.sleep(5)
table= driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[1]")
#print(table)

for row in table:
  print(row.text)


In [ ]:
tabla.head()

,Indice,Ultimo,dif,Maximo,Minimo,Fecha,Hora,Dif_año
0,IBEX 35®,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IBEX 35® con Dividendos,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IBEX MEDIUM CAP®,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IBEX SMALL CAP®,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IBEX 35® Bancos,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
driver = driversetup()
url = 'https://www.bolsasymercados.es/bme-exchange/es/Indices/Resumen'
driver.get(url)
time.sleep(5)

indices=[indice.text for indice in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[1]") ]
ultimos=[ultimo.text for ultimo in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[2]") ]
diferencias=[dif.text for dif in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[3]") ]
maximos=[maxim.text for maxim in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[4]") ]
minimos=[minim.text for minim in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[5]") ]
fechas=[fecha.text for fecha in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[6]") ]
horas=[hora.text for hora in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[7]") ]
diferencias_anyo=[dif_anyo.text for dif_anyo in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[8]") ]

In [ ]:
tabla=pd.DataFrame(columns=["Indice","Ultimo","dif","Maximo","Minimo","Fecha","Hora","Dif_año"])

tabla["Indice"]=indices
tabla["Ultimo"]=ultimos
tabla["dif"]=diferencias
tabla["Maximo"]=maximos
tabla["Minimo"]=minimos
tabla["Fecha"]=fechas
tabla["Hora"]=horas
tabla["Dif_año"]=diferencias_anyo

tabla.head()

,Indice,Ultimo,dif,Maximo,Minimo,Fecha,Hora,Dif_año
0,IBEX 35®,"9.451,00","0,29%","9.465,50","9.380,00",14/09/2023,13:18:53,"14,85%"
1,IBEX 35® con Dividendos,"31.225,80","0,29%","31.273,70","30.991,00",14/09/2023,13:18:53,"18,58%"
2,IBEX MEDIUM CAP®,"13.406,90","0,49%","13.407,70","13.344,60",14/09/2023,13:17:31,"4,75%"
3,IBEX SMALL CAP®,"7.862,00","-0,33%","7.905,20","7.833,40",14/09/2023,13:17:02,"9,42%"
4,IBEX 35® Bancos,"611,80","0,41%","613,00","604,20",14/09/2023,13:18:53,"16,87%"


In [ ]:
tabla.to_csv("Ejemplo_scraping1.csv", encoding='utf-8')